Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient resources are available for its entire duration.

The algorithm maintains a timeline to track resource usage over time and incrementally finds the first available time slot that satisfies all constraints.

Heuristics Employed:

* Job Prioritization: When multiple jobs are ready, the solver prioritizes jobs with the greatest total work content (duration * resource requirements). This aims to schedule more demanding tasks earlier.
* Earliest Feasible Start Time: The solver implicitly searches for the earliest time a job can begin by checking resource availability at the current time and advancing time based on resource releases.
* Mode Selection: (Not explicitly handled in this code, as it defaults to the first available mode if multiple existed.)

In [1]:
import json
import time
import heapq

def solve_rcmpsp(input_data):
    start_time_perf = time.time()
    
    # 1. Dynamic Data Extraction & Flattening
    # Based on your snippet: input_data['jobs'] is a list of projects, 
    # each containing a list of 'activities'.
    raw_projects = input_data.get("jobs", [])
    raw_resources = input_data.get("resources", [])
    
    # Map Resources to handle the positional list in 'resources_required'
    # We assume the order in 'resources_required' matches the order in 'resources'
    resource_ids = [r['id'] for r in raw_resources]
    resource_capacities = {r['id']: r['capacity'] for r in raw_resources}
    
    activities = {}
    in_degree = {}
    
    # Flatten projects into a single activity pool
    for project in raw_projects:
        for act in project.get("activities", []):
            act_id = act['id']
            # Convert positional resource list [0, 10, 0...] to {res_id: amount}
            req_list = act.get("resources_required", [])
            req_map = {resource_ids[i]: req_list[i] for i in range(len(req_list)) if req_list[i] > 0}
            
            activities[act_id] = {
                'duration': act['duration'],
                'successors': act['successors'],
                'requirements': req_map
            }
            in_degree[act_id] = 0

    # 2. Build Precedence Graph
    for act_id, info in activities.items():
        for succ in info['successors']:
            if succ in in_degree:
                in_degree[succ] += 1

    # Priority Rule: Greatest Total Work Content
    def get_priority(a_id):
        work = activities[a_id]['duration'] * sum(activities[a_id]['requirements'].values())
        return -work # Max-heap

    # 3. Scheduling Loop
    scheduled = {}
    finish_times = {}
    available_resources = resource_capacities.copy()
    resource_releases = []
    
    ready_queue = []
    for a_id, count in in_degree.items():
        if count == 0:
            heapq.heappush(ready_queue, (get_priority(a_id), a_id))

    current_time = 0
    total_activities = len(activities)

    while len(scheduled) < total_activities:
        can_schedule_any = False
        temp_queue = []

        # Sort ready queue to ensure we check highest priority first
        ready_queue.sort() 

        while ready_queue:
            prio, a_id = heapq.heappop(ready_queue)
            reqs = activities[a_id]['requirements']
            
            can_fit = all(available_resources.get(r_id, 0) >= amt for r_id, amt in reqs.items())
            
            if can_fit:
                scheduled[a_id] = current_time
                f_time = current_time + activities[a_id]['duration']
                finish_times[a_id] = f_time
                
                for r_id, amt in reqs.items():
                    available_resources[r_id] -= amt
                
                heapq.heappush(resource_releases, (f_time, a_id))
                can_schedule_any = True
            else:
                temp_queue.append((prio, a_id))
        
        ready_queue = temp_queue
        
        # Advance time
        if not can_schedule_any or not ready_queue:
            if resource_releases:
                next_event_time = resource_releases[0][0]
                current_time = max(current_time, next_event_time)
                
                while resource_releases and resource_releases[0][0] <= current_time:
                    _, finished_id = heapq.heappop(resource_releases)
                    for r_id, amt in activities[finished_id]['requirements'].items():
                        available_resources[r_id] += amt
                    
                    for succ in activities[finished_id]['successors']:
                        if succ in in_degree:
                            in_degree[succ] -= 1
                            if in_degree[succ] == 0:
                                heapq.heappush(ready_queue, (get_priority(succ), succ))
            else:
                break

    execution_time = (time.time() - start_time_perf)*1000
    return {
        "schedule": scheduled,
        "metrics": {
            "makespan": max(finish_times.values()) if finish_times else 0,
            "total_penalty": 0,
            "time": f"{execution_time:.4f}s"
        }
    }

In [2]:
with open("C:/Users/Admin/Desktop/mp_j90_a2_nr1.json", 'r') as f:
    data = json.load(f)
    
result = solve_rcmpsp(data)
print(json.dumps(result, indent=2))

{
  "schedule": {
    "1": 0,
    "4": 0,
    "2": 0,
    "3": 0,
    "6": 7,
    "10": 7,
    "16": 7,
    "19": 7,
    "7": 7,
    "17": 8,
    "40": 8,
    "53": 8,
    "28": 8,
    "32": 8,
    "5": 8,
    "20": 10,
    "13": 10,
    "25": 11,
    "39": 12,
    "45": 12,
    "11": 12,
    "9": 13,
    "18": 13,
    "57": 13,
    "8": 13,
    "14": 15,
    "23": 15,
    "29": 15,
    "36": 16,
    "27": 17,
    "35": 19,
    "21": 19,
    "22": 19,
    "49": 20,
    "31": 20,
    "26": 21,
    "30": 21,
    "54": 22,
    "47": 23,
    "15": 23,
    "12": 23,
    "42": 24,
    "41": 24,
    "34": 24,
    "43": 25,
    "33": 25,
    "44": 26,
    "51": 27,
    "58": 29,
    "38": 29,
    "63": 30,
    "60": 32,
    "46": 32,
    "50": 32,
    "52": 32,
    "56": 33,
    "24": 33,
    "48": 35,
    "55": 35,
    "64": 35,
    "59": 36,
    "67": 36,
    "65": 37,
    "83": 37,
    "81": 38,
    "61": 39,
    "37": 40,
    "70": 41,
    "72": 43,
    "66": 44,
    "76": 44,
    "88": 45